In [27]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce
from tqdm import tqdm

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2
index_path = os.path.join(os.getcwd(), 'Index_Storage')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
veto_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pm_full_tank_ODveto.h5'
odv_file = h5py.File(veto_path,'r')
odv_info = {}
for key in odv_file.keys():
    odv_info[key] = np.array(odv_file[key])

In [56]:
veto_dic = {}

pbar = ProgressBar(widgets=['Update Muon Values Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(odv_info['veto']))
pbar.start()
for i, veto in enumerate(odv_info['veto']):
    veto_dic[odv_info['root_files'][i], odv_info['event_ids'][i]] = veto
    pbar.update(i)
pbar.finish()

Update Muon Values Dictionary: 100% [000000000000000000000000000] Time: 0:00:17


In [4]:
np.where(np.array(list(veto_dic.values()))==1)[0].shape[0]

3130629

In [57]:
mv_path = '/fast_scratch/WatChMaL/data/michels.h5'

In [58]:
mv_file = h5py.File(mv_path,'r')
mv_info = {}
for key in mv_file.keys():
    mv_info[key] = np.array(mv_file[key])

In [59]:
pbar = ProgressBar(widgets=['Update Muon Values Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(mv_info['event_ids']))
pbar.start()
for i, is_michel in enumerate(mv_info['is_michel']):
    key = (mv_info['root_files'][i], mv_info['event_ids'][i])
    veto_dic[key] = 1 if (is_michel or veto_dic[key]==1) else 0
    pbar.update(i)
pbar.finish()

Update Muon Values Dictionary: 100% [000000000000000000000000000] Time: 0:00:14


In [11]:
np.where(np.array(list(veto_dic.values()))==1)[0].shape[0]

3172563

In [35]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')

dset_root_files = event_data_file['root_files']
dset_event_ids = event_data_file['event_ids']
dset_labels = event_data_file['labels']

In [65]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs']
train_idxs = idx_file['train_idxs']
val_idxs = idx_file['val_idxs']

In [14]:
test_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(test_idxs)])

100%|██████████| 3351020/3351020 [18:53<00:00, 2955.53it/s]


In [15]:
train_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(train_idxs)])

100%|██████████| 4188774/4188774 [23:45<00:00, 2938.90it/s]


In [16]:
val_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(val_idxs)])

100%|██████████| 837754/837754 [04:36<00:00, 3026.56it/s]


In [77]:
assert val_veto.shape[0] == val_idxs.shape[0]
assert train_veto.shape[0] == train_idxs.shape[0]
assert test_veto.shape[0] == test_idxs.shape[0]

In [18]:
post_veto_test_idxs=test_idxs[np.where(test_veto==0)[0]]
post_veto_train_idxs=train_idxs[np.where(train_veto==0)[0]]
post_veto_val_idxs=val_idxs[np.where(val_veto==0)[0]]

In [19]:
np.savez('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_post_veto_nomichel_pe_idxs.npz', train_idxs=post_veto_train_idxs, test_idxs=post_veto_test_idxs, val_idxs=post_veto_val_idxs)

In [20]:
event_data_file.close()

## Check

In [68]:
post_veto_idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_post_veto_nomichel_pe_idxs.npz',allow_pickle=True)
post_veto_test_idxs=post_veto_idx_file['test_idxs']
post_veto_train_idxs=post_veto_idx_file['train_idxs']
post_veto_val_idxs=post_veto_idx_file['val_idxs']

In [42]:
odv_dic = {}

pbar = ProgressBar(widgets=['Update Muon Values Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(odv_info['veto']))
pbar.start()
for i, veto in enumerate(odv_info['veto']):
    odv_dic[odv_info['root_files'][i], odv_info['event_ids'][i]] = veto
    pbar.update(i)
pbar.finish()

Update Muon Values Dictionary: 100% [000000000000000000000000000] Time: 0:00:17


In [33]:
mv_dic = {}

pbar = ProgressBar(widgets=['Update Muon Values Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(mv_info['is_michel']))
pbar.start()
for i, veto in enumerate(mv_info['is_michel']):
    mv_dic[mv_info['root_files'][i], mv_info['event_ids'][i]] = 1 if veto else 0
    pbar.update(i)
pbar.finish()

Update Muon Values Dictionary: 100% [000000000000000000000000000] Time: 0:00:07


In [45]:
for set_idxs in [post_veto_test_idxs, post_veto_val_idxs, post_veto_train_idxs]:
    pbar = ProgressBar(widgets=['Checking Set Vetoes: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(set_idxs))
    pbar.start()
    for i,idx in enumerate(set_idxs):
        pbar.update(i)
        key=(dset_root_files[idx], dset_event_ids[idx])
        assert odv_dic[key]==0
        if dset_labels[idx]==2:
            assert mv_dic[key]==0
    pbar.finish()

Checking Set Vetoes: 100% [0000000000000000000000000000000000000] Time: 0:15:05
Checking Set Vetoes: 100% [0000000000000000000000000000000000000] Time: 0:03:35
Checking Set Vetoes: 100% [0000000000000000000000000000000000000] Time: 0:17:51


In [66]:
len(test_idxs) + len(train_idxs)+len(val_idxs)

8377548

In [78]:
len(post_veto_test_idxs)+len(post_veto_val_idxs) + len(post_veto_train_idxs)+ np.where(val_veto==True)[0].shape[0] + np.where(test_veto==True)[0].shape[0]+np.where(train_veto==True)[0].shape[0]

8377548

In [82]:
assert len(post_veto_test_idxs) + np.where(test_veto==True)[0].shape[0] == len(test_idxs)
assert len(post_veto_val_idxs) + np.where(val_veto==True)[0].shape[0] == len(val_idxs)
assert len(post_veto_train_idxs) + np.where(train_veto==True)[0].shape[0] == len(train_idxs)